ref: https://www.kaggle.com/hamishdickson/bidirectional-lstm-in-keras-with-glove-embeddings

ref: https://lovit.github.io/nlp/representation/2018/09/05/glove/

# LOAD libs

In [1]:
import pandas as pd
import time
import os, gc

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.initializers import Constant
import re

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
%%time
train_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_train.csv")
test_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_test.csv")

CPU times: user 1.95 s, sys: 84.1 ms, total: 2.04 s
Wall time: 2.04 s


In [3]:
train_df.head()

,id,year_month,text,smishing,komo_morphs
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,XXX 은행 성산 XXX 팀장 이 ㅂ니다 . 행복 한 주말 되 시 어요
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,오늘 도 많이 웃 으시 는 하루 시작 하 시 어요 XXX 은행 지 ㄴ 월동 VIP ...
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,안녕하십니까 고객 님 . XXX 은행 이 ㅂ니다 . 금일 납부 하 시 어야 하 ㄹ ...
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,XXX 고객 님 안녕하세요 XXX 은행 XXX 지점 이 ㅂ니다 지나 ㄴ 한 하 아 ...
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,1월 은 새롭 ㅁ 이 가득 XXX 이 ㅂ니다 . 올 한하 아 더 많이 행복 하 ㄴ ...


In [4]:
pd.set_option('display.max_colwidth',-1)
train_df.head(20)

,id,year_month,text,smishing,komo_morphs
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,XXX 은행 성산 XXX 팀장 이 ㅂ니다 . 행복 한 주말 되 시 어요
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,오늘 도 많이 웃 으시 는 하루 시작 하 시 어요 XXX 은행 지 ㄴ 월동 VIP 라운지 XXX 올림
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니다.감사합니다. 새해 복 많이 받으십시오.XXX은행옥포XXX올림,0,안녕하십니까 고객 님 . XXX 은행 이 ㅂ니다 . 금일 납부 하 시 어야 하 ㄹ 금액 은 153600 원 이 ㅂ니다 . 감사 하 ㅂ니다 . 새해 복 많이 받 으시 ㅂ시오 . XXX 은행 옥포 XXX 올림
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 보내주신 성원에 감사드립니다. 설렘으로 시작한 2017년소망하시는 일 모두 이XXX 고객님의 가정에 늘 건강과 행복이 함께하길 기원하겠습니다. 사랑하는 가족과 함께 정을 나누는 행복한 설 명절 보내세요 XXX은행 XXX지점직원일동,0,XXX 고객 님 안녕하세요 XXX 은행 XXX 지점 이 ㅂ니다 지나 ㄴ 한 하 아 동안 저희 XXX 지점 에 보내 어 주 시 ㄴ 성원 에 감사 드리 ㅂ니다 . 설레 ㅁ 으로 시작 하 ㄴ 2017년 소망 하 시 는 일 모두 이 XXX 고객 님 의 가정 에 늘 건강 과 행복 이 함께 하 기 ㄹ 기원 하 겠 습니다 . 사랑 하 는 가족 과 함께 정 을 나누 는 행복 하 ㄴ 설 명절 보내 시 어요 XXX 은행 XXX 지점 직원 일동
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,1월 은 새롭 ㅁ 이 가득 XXX 이 ㅂ니다 . 올 한하 아 더 많이 행복 하 ㄴ 한해 되 시 기 ㄹ 바라 ㅂ니다
5,6,2017-01,행복한주말보내세요XXX용현남전담직원대리 XXX올림,0,행복 하 ㄴ 주말 보 내세 요 XXX 용 현 남 전담 직원 대리 XXX 올림
6,7,2017-01,XXX 고객님 안녕하세요XXX은행 무교지점 XXX과장입니다 오늘 아침에 눈을 뜨니 눈이 많이 와서 온 세상이 하얗게 변했더군요어릴적에는 눈이 마냥 좋기만 했는데눈을 보는 순간 좋기보다는 먼저. 출근을 걱정하게 되더라구요이제 진정한 어른이 되었음을 느끼게 됩니다새해 인사를 드리기전에..제가 발령이 나서 본점으로 가게되었어요ㅠㅠ. 진정한 구정 새해인사보다 이별인사를 먼저 드리게 되어 송구할 뿐입니다그동안 보내주신 성원에 감사.또 감사드리며XXX은행에 있는XXX고객님을 평생 저의 고객으로 모시겠습니다.새해 복 많이 받으시고 2017년 원하고 바라시는 모든일 이루시길 기원합니다XXX은행 무교지점 XXX과장올림,0,XXX 고객 님 안녕하세요 XXX 은행 무교 지점 XXX 과장 이 ㅂ니다 오늘 아침 에 눈 을 뜨 니 눈 이 많이 오 아서 오 ㄴ 세상 이 하얗 게 변하 았 더군요 어리 ㄹ 적 에 는 눈 이 마냥 좋 기 만 하 았 는데 눈 을 보 는 순간 좋 기 보다 는 먼저 . 출근 을 걱정 하 게 되 더라구요 이제 진정 하 ㄴ 어른 이 되 었 음 을 느끼 게 되 ㅂ니다 새해 인사 를 드리 기 전 에 .. 제가 발령 이 나서 어 본점 으로 가게되었어요ㅠㅠ. 진정 하 ㄴ 구정 새 해인사 보다 이별 인사 를 먼저 드리 게 되 어 송구 하 ㄹ 뿐 이 ㅂ니다 그동안 보내 어 주 시 ㄴ 성원 에 감사 . 또 감사 드리 며 XXX 은행 에 있 는 XXX 고객 님 을 평생 저 의 고객 으로 모시 겠 습니다 . 새해 복 많이 받 으시 고 2017 년 원하 고 바라 시 는 모든 일 이 루시 길 기원 하 ㅂ니다 XXX 은행 무교 지점 XXX 과 장 올림
7,8,2017-01,XXX 고객님지난 한해 베풀어 주신 은혜 진심으로 감사 드립니다.가슴 깊이 간직 하겠습니다.정유년 새해 가족 모두 행복하시고 뜻 하는 바 모두이루시길 진심으로 소망합니다XXX XXX은행 XXX 지점장 정 XXX 배상,0,XXX 고객 님 지나 ㄴ 한하 아 베풀 어 주 시 ㄴ 은혜 진심 으로 감사 드리 ㅂ니다 . 가슴 깊이 간직 하 겠 습니다 . 정유 년 새해 가족 모두 행복 하 시 고 뜻 하 는 바 모두 이루 시 기 ㄹ 진심 으로 소망 하 ㅂ니다 XXX XXX 은행 XXX 지점장 정 XXX 배상
8,9,2017-01,설연휴 가족들과 훈훈한 정 나누시고 정겨운추억 많이 만드세요XXX오XXX올림,0,설 연휴 가족 들 과 훈훈 하 ㄴ 정 나누 시 고 정겨운 추억 많이 만들 시 어요 XXX 오 XXX 올림
9,10,2017-01,(광고)XXXBaXXX고객님들 뒤엔XXX 언제나 XXX새로운 마음가짐으로 새롭게 준비합니다.당행상품의 자격기준과 심사기준이 완화되어 상품에 대해 간단하게 상품정보 전달드립니다.수신을 희망하지 않으실 경우에는 거부 라는 답장을 주시면 KISA 수신거부 목록에 등록을 시켜 두번 다시 발송되지 않도록 조치를 취해드리겠습니다.신청방법: XXX-XXX-XXX 터치하셔서 문자로 상담 또는 00시 00분 상담이라고 답장을 보내주시거나 전화를 주시면 친절하고 안전한 상담으로 도움드리겠습니다.카카오톡 상담서비스 시행중 citibankloan친구추가 후 상담신청이런 분들께 권해드립니다.1. 시중은행권에서 기존한도를 모두 사용 중이신 분.2. 기대출의 원리금 균등방식으로 인해 월 불입금액이 부담스러우신 분.3. 총 채무금액은 작으나 채무건수가 많아서 관리가 안 되시는 XXX고객님들께 말씀드립니다. 높은 금리 때문에 힘들다고 느끼셨을 때 대출이 필요하실 때 언제나XXX 고객님들 뒤에서 든든한 파트너로서 도와드리겠습니다.당행 상품특징1. 당행 거래가 없으셔도 진행가능.2. 시중은행권보다 높은 한도.3. 자체 등급으로 판단하므로 낮은신용자도 진행가능.4. 부채가 많다면 부채 통합상품으로 전환가능.5. 원리금 균등방식을 이자만 납입하는 만기일시로 전환가능.6. 가상 조회를 통해 한도 및 금리를 한 번에 확인가능.당행 상품안내1. 직장인전용 신용상품한도: 최대 1억 4000만 원까지.금리: 최저 2.97%자격: XXX은행 자체 등급과 기업 리스트에 따라 차등적용.2. 채무통합 전환상품한도: 연봉 대비 300%까지.금리: 최저 2.97%자격: XXX은행 자체 등급과 기업 리스트에 따라 차등적용.3. 새 희망 홀씨한도: 최대 2500만 원.금리: 8.68%12%자격: 연봉 4000만 원 이하 고객 대상으로 XXX 자체 등급에 따라(광고)XXX,1,( 광고 ) XXXBaXXX 고객 님 들 뒤 엔 XXX 언제나 XXX 새롭 ㄴ 마음가짐 으로 새롭 게 준비 하 ㅂ니다 . 당행 상품 의 자격 기준 과 심사 기준 이 완화 되 어 상품 에 대하 아 간단 하 게 상품 정보 전달 드리 ㅂ니다 . 수신 을 희망 하 지 않 으시 ㄹ 경우 에 는 거부 이 라는 답장 을 주 시 면 KISA 수신 거부 목록 에 등록 을 시키 어 두 번 다시 발송 되 지 않 도록 조치 를 취하 아 드리 겠 습니다 . 신청 방법 : XXX - XXX - XXX 터치 하 시 어서 문자 로 상담 또는 00 시 00 분 상담 이 라고 답장 을 보내 어 주 시 거나 전화 를 주 시 면 친절 하 고 안전 하 ㄴ 상담 으로 도움 드리 겠 습니다 . 카카오톡 상담 서비스 시행 중 citibankloan 친구 추가 후 상담 신청 이런 분 들 께 권하 아 드리 ㅂ니다 . 1 . 시중 은행 권 에서 기존 한도 를 모두 사용 중 이신 분 . 2 . 기 대출 의 원리금 균등 방식 으로 인해 월 불 입금액 이 부담 스럽 시 ㄴ 분 . 3 . 총 채무 금액 은 작 으나 채무 건수 가 많 아서 관리 가 안 되 시 는 XXX 고객 님 들 께 말씀 드리 ㅂ니다 . 높 은 금리 때문 에 힘들 다고 느끼 시 었 을 때 대출 이 필요 하 시 ㄹ 때 언제 나 XXX 고객 님 들 뒤 에서 든든 하 ㄴ 파트너 로서 돕 아 드리 겠 습니다 . 당행 상품 특징 1 . 당행 거래 가 없으셔도 진행 가능 . 2 . 시중 은행 권 보다 높 은 한도 . 3 . 자체 등급 으로 판단 하 므

In [5]:
train_df['l'] = train_df['komo_morphs'].apply(lambda x: len(str(x).split(' ')))
print('mean length of data : {}'.format(train_df['l'].mean()))
print('max length of data : {}'.format(train_df['l'].max()))
print('std length of data : {}'.format(train_df['l'].std()))

mean length of data : 89.27805504401155
max length of data : 644
std length of data : 114.02346925047749


In [6]:
test_df['l'] = test_df['komo_morphs'].apply(lambda x: len(str(x).split(' ')))
print('mean length of data : {}'.format(test_df['l'].mean()))
print('max length of data : {}'.format(test_df['l'].max()))
print('std length of data : {}'.format(test_df['l'].std()))

mean length of data : 152.6691266912669
max length of data : 565
std length of data : 116.83732804508064


---
### Sampling for Debug

In [7]:
df_0 = train_df[train_df['smishing']==0].sample(frac=1)
df_1 = train_df[train_df['smishing']==1].sample(frac=1)

# balancing the training
sample_size = min(len(df_0),len(df_1))

data = pd.concat([df_0.head(sample_size), df_1.head(sample_size)]).sample(frac=1)

In [8]:
data.shape

(37406, 6)

In [9]:
data.head()

,id,year_month,text,smishing,komo_morphs,l
255687,290195,2018-08,XXX 광고 문자입니다 (광고) XXX 항상 저희 XXX을 이용해주셔서 감사합니다. 2018년 하반기 정부지원 상품이 개편되어 안내드립니다. 금년 좋은혜택으로 저렴하게 이용하실수 있는 마지막기회입니다. 고민하지 마시고 전화주세요 항상 이용해주셔서 감사합니다 (내용) 상품안내 상품명 XXX드림loan새희망홀씨loan 본상품은 서민지원상품으로 한국자산관리공사(KEMCO)에서 지원하고 XXX 에서 판매하는 상품입니다. 상품특징 1. 상품진행시 별도 발생하는 금액 없이 진행가능! 2. 개인사업자 등급상관없이! 3. 최고 9000 만원 까지! 4. 최저 3.26.9개인등급에따라 차등적용 5. 별도 추가비용없이 없이 자유롭게 납부가능! 6. 기존 타사이용중인 상품낮은상품으로 전환가능 자격조건 1.만21세65세 2.근로소득자사업자프리랜서주부학생 3.최근거절되신분 기존타사 이용중이신 분들도가능 신청기간 9월12일마감. 접수자가 많으니 빠른신청 바랍니다. 신청방법 XXX-XXX-XXX전화 하셔서 1번 누르신후 필요금액 입력하시면 접수완료 신청자가 많아 상담이 지연될수 XXX,1,XXX 광고 문자 이 ㅂ니다 ( 광고 ) XXX 항상 저희 XXX 을 이용 하 아 주 시 어서 감사 하 ㅂ니다 . 2018 년 하반기 정부 지원 상품 이 개편 되 어 안내 드리 ㅂ니다 . 금년 좋 은 혜택 으로 저렴 하 게 이용하 실수 있 는 마지막 기회 이 ㅂ니다 . 고민 하 지 마시 고 전화 주 시 어요 항상 이용 하 아 주 시 어서 감사 하 ㅂ니다 ( 내용 ) 상품 안내 상품명 XXX 드림 loan 새 희망 홀 씨 loan 보 ㄴ 상품 은 서민지 원 상품 으로 한국자산관리공사 ( KEMCO ) 에서 지원 하 고 XXX 에서 판매 하 는 상품 이 ㅂ니다 . 상품 특징 1 . 상품 진행 시 별도 발생 하 는 금액 없이 진행 가능 ! 2 . 개인 사업자 등급 상관없이 ! 3 . 최고 9000 만원 까 지 ! 4 . 최저 3 . 26 . 9 개인 등급 에 따라 차등 적용 5 . 별도 추가 비용 없이 없이 자유 롭 게 납부 가능 ! 6 . 기존 타사 이용 중인 상품 낮 은 상품 으로 전환 가능 자격 조건 1 . 만 21 세 65 세 2 . 근로 소득자 사업자 프리랜서 주부 학생 3 . 최근 거절 되 시 ㄴ 분 기존 타사 이용 중 이신 분 들 도 가능 신청 기간 9 월 12 일 마감 . 접수 자 가 많 으니 빠르 ㄴ 신청 바라 ㅂ니다 . 신청 방법 XXX - XXX - XXX 전화 하 시 어서 1 번 누르 시 ㄴ 후 필요 금액 입력 하 시 면 접수 완료 신청자 가 많 아 상담 이 지연 되 ㄹ 수 XXX,272
262983,298405,2018-09,XXX 고객님민족의 명절 추석이 다가오고 있습니다.추석을 글자대로 풀이하면 가을의 달빛이 가장 좋은 밤이라고 합니다.가을의 가장 좋은 날 반가운 얼굴을 만나며 충분한 쉼이 있는 시간 되세요 (-)XXX은행 송도역지점 XXX 올림,0,XXX 고객 님 민족 의 명절 추석 이 다가오 고 있 습니다 . 추석 을 글자 대로 풀이 하 면 가을 의 달빛 이 가장 좋 은 밤 이 라고 하 ㅂ니다 . 가을 의 가장 좋은 날 반갑 ㄴ 얼굴 을 만나 며 충분 하 ㄴ 쉬 ㅁ 이 있 는 시간 되 시 어요 ( - ) XXX 은행 송도역 지점 XXX 올림,65
122787,142087,2017-07,(광고)XXX따뜻한 마음 XXX날씨와 다르게 항상 얼어붙은 경기침XXX고객님들의 마음을 녹일 수 있는 XXX.상품에 대해 간단하게 정보를 전달드립니다.똑똑한 신용관리 결코 어렵지않습니다. 본인의 신용은 스스로 직접 관리해야 하는 시대입니다. 최소한의 지식과 상식은 알고 계셔야 금전적인 손해 보는 일이 없습니다.신청방법: XXX-XXX-XXX 터치하셔서 문자로 상담 또는 00시 00분 상담이라고 답장을 보내주시거나 전화를 주시면 친절하고 안전한 상담으로 도움드리겠습니다.카카오톡 상담서비스 시행중 citibankloan친구추가 후 상담신청수신을 희망하지 않으실 경우에는 거부 라는 답장을 주시면 KISA 수신거부 목록에 등록을 시켜 두번 다시 발송되지 않도록 조치를 취해드리겠습니다.당행상품의 특징1. 당행 거래가 없으셔도 진행가능2. 시중은행권보다 높은 한도3. 부채가 많다면 부채 통합상품으로 전환가능4. 원리금 균등방식을 이자만 납입하는 만기일시로 전환가능5. 가상 조회를 통해 한도 및 금리를 한 번에 확인가능당행상품의 조건안내1. 직장인전용 신용상품한도: 최대 1억 4000만 원까지금리: 최저 2.97%자격: XXX은행 자체 등급과 기업 리스트에 따라 차등적용2. 채무통합 전환상품한도: 연봉 대비 300%까지금리: 최저 2.97%자격: XXX은행 자체 등급과 기업 리스트에 따라 차등적용3. 새 희망 홀씨한도: 최대 3000만 원금리: 8.68%12%자격: 연봉 4500만 원 이하 고객 대상으로 당사 자체 등급에 따라 차등적용이런 분들께 권해드립니다.1. 시중은행권에서 기존한도를 모두 사용 중이신 분2. 기대출의 원리금 균등방식으로 인해 월 불입금액이 부담스러우신 분3. 총 채무금액은 작으나 채무건수가 많아서 관리가 안 되시는 분신청방법: XXX-XXX-XXX 터치하셔서 문자로 상담 또는 00시 00분 상담이라고 답장을 보내주시거나 전화를 주시면 친절하고 안전한 상담으로 도움드리겠습니다.카카오톡 상담서비스 시행중 citibankloan친구추가 후 상담신청수신을 희망하지 않으실 경우에는 거부 라는 답장을 주시면 KISA 수신거부 목록에 등록을 시켜 두번 다시 발송되지 않도록 조치를 취해드리겠습니다.항상 감사드립니다.준법감시인 심의필XXX-XXX-XXX6-LMS(광고)XXX,1,( 광고 ) XXX 따뜻 하 ㄴ 마음 XXX 날씨 와 다르 게 항상 얼어붙 은 경 기침 XXX 고객 님 들 의 마음 을 녹이 ㄹ 수 있 는 XXX . 상품 에 대하 아 간단 하 게 정보 를 전달 드리 ㅂ니다 . 똑똑 하 ㄴ 신용 관리 결코 어렵 지 않 습니다 . 본인 의 신용 은 스스로 직접 관리 하 아야 하 는 시대 이 ㅂ니다 . 최소한 의 지식 과 상식 은 알 고 계 시 어야 금전 적 이 ㄴ 손해 보 는 일 이 없 습니다 . 신청 방법 : XXX - XXX - XXX 터치 하 시 어서 문자 로 상담 또는 00 시 00 분 상담 이 라고 답장 을 보내 어 주 시 거나 전화 를 주 시 면 친절 하 고 안전 하 ㄴ 상담 으로 도움 드리 겠 습니다 . 카카오톡 상담 서비스 시행 중 citibankloan 친구 추가 후 상담 신청 수신 을 희망 하 지 않 으시 ㄹ 경우 에 는 거부 이 라는 답장 을 주 시 면 KISA 수신 거부 목록 에 등록 을 시키 어 두 번 다시 발송 되 지 않 도록 조치 를 취하 아 드리 겠 습니다 . 당행 상품 의 특징 1 . 당행 거래 가 없으셔도 진행 가능 2 . 시중 은행 권 보다 높 은 한도 3 . 부채 가 많 다면 부채 통합 상품 으로 전환 가능 4 . 원리금 균등 방식 을 이자 만 납입 하 는 만기일 시로 전환 가능 5 . 가상 조회 를 통하 아 한도 및 금리 를 한 번 에 확인 가능 당행 상품 의 조건 안내 1 . 직장인 전용 신용 상품 한도 : 최대 1 억 4000 만 원 까지 금리 : 최저 2 . 97 % 자격 : XXX 은행 자체 등급 과 기업 리스트 에 따르 아 차등 적용 2 . 채무 통합 전환 상품 한도 : 연봉 대비 300 % 까지 금리 : 최저 2 . 97 % 자격 : XXX 은행 자체 등급 과 기업 리스트 에 따르 아 차등 적용 3 . 새 희망 홀 씨 한도 : 최대 3000 만 원 금리 : 8 . 68 % 12 % 자격 : 연봉 4500 만 원 이하 고객 대상 으로 당사 자체 등급 에 따르 아 차등 적용 이렇 ㄴ 분 들 께 권하 아 드리 ㅂ니다 . 1 . 시중 은행 권 에서 기존 한도 를 모두 사용 중 이신 분 2 . 기 대출 의 원리금 균등 방식 으로 인해 월 불 입금액 이 부담 스럽 시 ㄴ 분 3 . 총 채무 금액 은 작 으나 채무 건수 가 많 아서 관리 가 안 되 시 는 분 신청 방법 : XXX - XXX - XXX 터치 하 시

In [10]:
data['l'] = data['komo_morphs'].apply(lambda x: len(str(x).split(' ')))
print('mean length of data : {}'.format(data['l'].mean()))
print('max length of data : {}'.format(data['l'].max()))
print('std length of data : {}'.format(data['l'].std()))

mean length of data : 241.89154146393628
max length of data : 637
std length of data : 199.44390427547762


## Train GloVe

### MAKE Glove input by soynlp

In [17]:
result = list(data['komo_morphs'].values)

In [18]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.vectorizer import sent_to_word_contexts_matrix

#corpus_path = '2016-10-20_article_all_normed_ltokenize.txt'
#corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True)

x, idx2vocab = sent_to_word_contexts_matrix(
    result,
    windows=3,
    min_tf=10,
    tokenizer=lambda x:x.split(), # (default) lambda x:x.split(),
    dynamic_weight=True,
    verbose=True)

print(x.shape) # (36002, 36002)

Create (word, contexts) matrix
  - counting word frequency from 37405 sents, mem=0.680 Gb
  - scanning (word, context) pairs from 37405 sents, mem=0.718 Gb
  - (word, context) matrix was constructed. shape = (5290, 5290)                    
  - done
(5290, 5290)


In [21]:
from glove import Corpus, Glove

In [23]:
glove = Glove(no_components=200, learning_rate=0.05)
glove.fit(x.tocoo(), epochs=5, no_threads=4, verbose=True)

Performing 5 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [27]:
dictionary = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
glove.add_dictionary(dictionary)

In [28]:
glove.dictionary

{'.': 0,
 '하': 1,
 '이': 2,
 'XXX': 3,
 '시': 4,
 'ㅂ니다': 5,
 '을': 6,
 'ㄴ': 7,
 '는': 8,
 ')': 9,
 '(': 10,
 '아': 11,
 '드리': 12,
 '-': 13,
 '를': 14,
 '가능': 15,
 '%': 16,
 '상품': 17,
 '에': 18,
 '어': 19,
 '되': 20,
 '으로': 21,
 ':': 22,
 '대출': 23,
 '은': 24,
 '의': 25,
 '습니다': 26,
 '분': 27,
 '금리': 28,
 '상담': 29,
 '있': 30,
 '주': 31,
 '고': 32,
 '1': 33,
 '은행': 34,
 '2': 35,
 '가': 36,
 '로': 37,
 '한도': 38,
 '3': 39,
 'ㄹ': 40,
 '면': 41,
 '등급': 42,
 '신청': 43,
 '까지': 44,
 '지': 45,
 '겠': 46,
 '들': 47,
 '고객': 48,
 '통합': 49,
 '!': 50,
 '게': 51,
 '진행': 52,
 '부채': 53,
 '4': 54,
 '만': 55,
 '과': 56,
 '도': 57,
 '수': 58,
 '거부': 59,
 '직장인': 60,
 '및': 61,
 '이자': 62,
 '으시': 63,
 '안내': 64,
 '문자': 65,
 '신용': 66,
 '님': 67,
 '전화': 68,
 '에서': 69,
 '않': 70,
 '전환': 71,
 '광고': 72,
 '어요': 73,
 '신용등급': 74,
 '방식': 75,
 '방법': 76,
 '수신': 77,
 '많': 78,
 '추가': 79,
 '기': 80,
 '이상': 81,
 '사용': 82,
 '만원': 83,
 '권': 84,
 '월': 85,
 '자격': 86,
 '금융': 87,
 '적용': 88,
 '6': 89,
 '거나': 90,
 '자금': 91,
 '채무': 92,
 '보': 93,
 '금액': 94,
 '이용': 9

In [30]:
glove.word_vectors.shape

(5290, 200)

In [33]:
glove.word_vectors[:10]

array([[ 0.00361557,  0.29555459, -0.55063571, ...,  0.1405368 ,
        -0.09002174, -0.43244135],
       [ 0.36841672,  0.21683283, -0.46029392, ...,  0.29787018,
         0.0506387 ,  0.2510601 ],
       [-0.18082537,  0.27334548, -0.50357991, ...,  0.27228597,
         0.1207886 , -0.29229095],
       ...,
       [ 0.1387997 ,  0.04615299, -0.42753883, ...,  0.51486653,
         0.29129282, -0.02860357],
       [ 0.31697143, -0.1455581 , -0.37248333, ..., -0.23284892,
        -0.18883817, -0.21061272],
       [ 0.10200921,  0.32111536, -0.38250703, ..., -0.18303097,
        -0.28857173,  0.29670849]])

In [37]:
ga = glove.word_vectors
gd = glove.dictionary

In [106]:
import numpy as np

In [107]:
glove_embedding = pd.DataFrame()
glove_embedding['key'] = list(gd.keys())
glove_embedding['vectors'] = list(ga)
glove_embedding.head()

,key,vectors
0,.,"[0.003615566020709758, 0.2955545932336471, -0.550635710258467, -0.03659935987304708, 0.35635844212855355, 0.20456228560700299, -0.12523244208227655, -0.235439594608491, -0.41725914930634883, -0.11409596204256091, 0.15277828041585423, 0.25015255086091226, 0.2972712740946616, 0.0529258559979769, 0.42705007782609256, 0.0893466934884387, 0.3521808522955247, -0.1815190330062403, -0.4857134774356654, 0.16111810453051598, -0.3397282309990206, -0.49495325527710726, 0.37254306830391476, 0.30886324842006874, 0.48137272929395747, 0.21480347182921453, -0.016503603707838585, 0.19114487230374047, 0.5902186324601142, 0.3129316493201005, 0.149252492673734, 0.5209685066760917, 0.08517352956897527, 0.11710674703767258, -0.406082576368318, -0.45082008616434066, 0.13103788877153433, -0.29806723307688787, -0.28478348059355246, 0.2101685368726635, 0.2711793928031441, -0.36313611708770327, 0.14367483695360975, -0.3347164661911093, -0.5022575232523055, 0.5270329988966629, 0.08325252619245907, 0.07372005286271699, 0.1258936905789908, -0.3401605522763126, -0.3758003291229286, 0.027310803691368686, -0.23592762398317862, -0.3560212036860074, -0.08874439082467277, -0.03453941058630844, -0.353626540187095, 0.060544554197044166, 0.38004157501729324, 0.2994956882174277, 0.08613934014464747, 0.2473279833632801, 0.27862750565555244, -0.3209019762324416, 0.3342466708452471, 0.0012212710590514774, 0.22799900946625035, 0.36825372913561705, -0.047306359692787965, -0.47543984965551594, 0.31620875506637003, -0.14726210909621001, -0.2266503404804286, -0.36465882230210966, -0.17666843813086555, -0.3639209907153031, -0.01600517227764092, 0.36919966151734773, 0.1500001190395796, -0.45161460731355807, 0.32697005162687176, 0.37651585817277805, -0.3250921751111267, 0.39219213385049645, 0.05086442513820652, -0.3808335584916716, 0.24129153579215487, -0.05038088897433309, 0.0020319861012805347, -0.4287431289997003, 0.1253688898527179, -0.3421234982519863, -0.1522159386142391, -0.45469200240785446, -0.026560342098869653, -0.022183656548522095, -0.2560305066601014, -0.22494669514246068, -0.07867989345262302, -0.5343060872772812, ...]"
1,하,"[0.3684167223279421, 0.21683283339963882, -0.46029391527714286, 0.613362949841267, -0.01854308915542202, 0.2723467804420542, -0.3100270656340079, -0.3635900790097026, -0.3792143188522416, -0.2939210740101969, 0.2805839557466231, 0.19045134845655934, -0.12515747486941228, -0.4776413214588977, 0.20175522374248794, -0.046227225557636455, 0.7243382728622005, 0.08701520320335254, 0.3209949458444391, 0.3927745501030554, -0.13883130866076263, 0.047843856745089064, 0.009089304677323872, 0.004698468697341124, 0.4556523342512656, -0.15819907608391692, 0.23329637653560087, 0.29061281046741316, 0.520769281209353, -0.30699804520666385, 0.0868463373972373, 0.42925093365993444, 0.06331167952592757, -0.1619245797428304, -0.21535014889009232, -0.3645332118252147, 0.13309419154096988, 0.18902064239600283, -0.2992064755660503, -0.3163266283042527, -0.05536579791200373, -0.569132891139147, -0.1780977800380564, -0.13325399518763964, 0.25182259887198993, 0.27998031437719006, 0.20979745408526132, -0.21267827354548244, 0.114480840564232, -0.03507353950864374, 0.4688946994462761, 0.07649189637109959, -0.08578378492620081, 0.5060412655087219, -0.23645624000755952, -0.3139573506700595, 0.28556740956563864, -0.38580963071341606, 0.052349399913276626, -0.07794430829021712, 0.12492693793517162, 0.23989631554328633, 0.0860344542307886, -0.4179812344986369, 0.14097294245140948, 0.5757644674801705, -0.388642957812839, 0.3660038899813664, -0.14548735864169227, 0.24537952509754524, 0.3882425989938473, -0.26565139881326943, -0.4113715077192411, -0.38671694264955797, 0.10722914564827382, -0.21550250143111352, 0.23774053337325216, -0.08541730914652682, 0.14438232657836977, -0.3615782399773897, -0.2906303432863967, 0.2151590316066342, -0.5652863184670917, 0.4335398979888103, 0.3384114240146255, 0.28727120256879835, -0.07353999072101625, -0.017097137605911528, 0.2

In [108]:
glove_embedding['vectors'] = glove_embedding['vectors'].map(str)

In [110]:
glove_embedding['vectors'] = glove_embedding['key'] + glove_embedding['vectors']

In [112]:
glove_embedding.head()

,key,vectors
0,.,.[ 0.00361557 0.29555459 -0.55063571 -0.03659936 0.35635844 0.20456229\n -0.12523244 -0.23543959 -0.41725915 -0.11409596 0.15277828 0.25015255\n 0.29727127 0.05292586 0.42705008 0.08934669 0.35218085 -0.18151903\n -0.48571348 0.1611181 -0.33972823 -0.49495326 0.37254307 0.30886325\n 0.48137273 0.21480347 -0.0165036 0.19114487 0.59021863 0.31293165\n 0.14925249 0.52096851 0.08517353 0.11710675 -0.40608258 -0.45082009\n 0.13103789 -0.29806723 -0.28478348 0.21016854 0.27117939 -0.36313612\n 0.14367484 -0.33471647 -0.50225752 0.527033 0.08325253 0.07372005\n 0.12589369 -0.34016055 -0.37580033 0.0273108 -0.23592762 -0.3560212\n -0.08874439 -0.03453941 -0.35362654 0.06054455 0.38004158 0.29949569\n 0.08613934 0.24732798 0.27862751 -0.32090198 0.33424667 0.00122127\n 0.22799901 0.36825373 -0.04730636 -0.47543985 0.31620876 -0.14726211\n -0.22665034 -0.36465882 -0.17666844 -0.36392099 -0.01600517 0.36919966\n 0.15000012 -0.45161461 0.32697005 0.37651586 -0.32509218 0.39219213\n 0.05086443 -0.38083356 0.24129154 -0.05038089 0.00203199 -0.42874313\n 0.12536889 -0.3421235 -0.15221594 -0.454692 -0.02656034 -0.02218366\n -0.25603051 -0.2249467 -0.07867989 -0.53430609 -0.30063394 0.22210178\n 0.15167214 -0.11587943 0.14877169 0.025182 -0.11758429 -0.33462337\n 0.2778078 0.25735238 -0.48218057 -0.25686212 -0.26101231 0.32408123\n 0.08470069 -0.21274233 -0.21734172 0.00139867 0.15999863 -0.06655995\n 0.22212123 -0.25234552 0.17242752 0.33072345 -0.48437418 -0.07916299\n 0.18509961 -0.06689423 0.1527958 -0.16343994 -0.24883281 0.33020765\n -0.01695553 -0.64218903 -0.35164878 0.10997594 0.16636878 0.24044841\n 0.08596227 0.41851186 0.47105082 0.27110334 -0.18742204 -0.34662244\n -0.19053041 0.20188182 -0.027384 -0.36669937 -0.2779636 -0.40238442\n -0.4008954 -0.31187053 -0.05521224 0.18963317 -0.11824411 -0.37755248\n 0.19430119 -0.35738383 0.41514174 0.0987715 0.09852836 -0.28925388\n -0.04769074 0.26854134 0.11288332 -0.3396557 -0.1980434 -0.15014548\n -0.31946746 -0.16966036 0.00285232 0.31716843 0.49169938 -0.16937593\n -0.03208759 -0.12566696 0.13767718 0.00668406 0.34061395 -0.30574728\n -0.4786323 0.2927631 0.04892921 0.30963658 0.00709893 0.17760978\n -0.43836653 0.56808684 0.53985977 0.01376702 0.09184149 0.21105069\n 0.47836747 -0.65359274 -0.13576892 0.28128686 -0.22935518 0.1405368\n -0.09002174 -0.43244135]
1,하,하[ 0.36841672 0.21683283 -0.46029392 0.61336295 -0.01854309 0.27234678\n -0.31002707 -0.36359008 -0.37921432 -0.29392107 0.28058396 0.19045135\n -0.12515747 -0.47764132 0.20175522 -0.04622723 0.72433827 0.0870152\n 0.32099495 0.39277455 -0.13883131 0.04784386 0.0090893 0.00469847\n 0.45565233 -0.15819908 0.23329638 0.29061281 0.52076928 -0.30699805\n 0.08684634 0.42925093 0.06331168 -0.16192458 -0.21535015 -0.36453321\n 0.13309419 0.18902064 -0.29920648 -0.31632663 -0.0553658 -0.56913289\n -0.17809778 -0.133254 0.2518226 0.27998031 0.20979745 -0.21267827\n 0.11448084 -0.03507354 0.4688947 0.0764919 -0.08578378 0.50604127\n -0.23645624 -0.31395735 0.28556741 -0.38580963 0.0523494 -0.07794431\n 0.12492694 0.23989632 0.08603445 -0.41798123 0.14097294 0.57576447\n -0.38864296 0.36600389 -0.14548736 0.24537953 0.3882426 -0.2656514\n -0.41137151 -0.38671694 0.10722915 -0.2155025 0.23774053 -0.08541731\n 0.14438233 -0.36157824 -0.29063034 0.21515903 -0.56528632 0.4335399\n 0.33841142 0.2872712 -0.07353999 -0.01709714 0.25221327 -0.39526921\n -0.47889734 0.00145772 -0.30913501 -0.16853361 -0.097773 0.12499947\n -0.07375591 -0.13768481 0.10251603 -0.55976113 -0.31442303 0.17463187\n 0.18294199 0.12898277 0.25978554 -0.06468903 0.16384478 -0.26891798\n 0.17706376 0.29710707 -0.39470815 0.16247905 -0.21252991 0.26779443\n 0.48325771 -0.02785288 0.21217176 -0.02158545 -0.43185398 0.33519644\n 0.20739244 -0.3321114 0.17731226 0.30283018 -0.35531105 -0.27099341\n -0.0395523 -0.16268168 -0.02659771 0.15538934 -0.34828881 0.21435116\n -0.35870617 -0.54089611 -0.39327171 -0.09843882 -0.12970032 0.21336714\n 0.34533007 -0.005

In [113]:
for i in tqdm(range(len(glove_embedding))):
    glove_embedding['vectors'][i] = glove_embedding['vectors'][i].replace('[','').replace(']','').replace('\n','')

100%|██████████| 5290/5290 [00:00<00:00, 11950.04it/s]


In [114]:
glove_embedding.head()

,key,vectors
0,.,. 0.00361557 0.29555459 -0.55063571 -0.03659936 0.35635844 0.20456229 -0.12523244 -0.23543959 -0.41725915 -0.11409596 0.15277828 0.25015255 0.29727127 0.05292586 0.42705008 0.08934669 0.35218085 -0.18151903 -0.48571348 0.1611181 -0.33972823 -0.49495326 0.37254307 0.30886325 0.48137273 0.21480347 -0.0165036 0.19114487 0.59021863 0.31293165 0.14925249 0.52096851 0.08517353 0.11710675 -0.40608258 -0.45082009 0.13103789 -0.29806723 -0.28478348 0.21016854 0.27117939 -0.36313612 0.14367484 -0.33471647 -0.50225752 0.527033 0.08325253 0.07372005 0.12589369 -0.34016055 -0.37580033 0.0273108 -0.23592762 -0.3560212 -0.08874439 -0.03453941 -0.35362654 0.06054455 0.38004158 0.29949569 0.08613934 0.24732798 0.27862751 -0.32090198 0.33424667 0.00122127 0.22799901 0.36825373 -0.04730636 -0.47543985 0.31620876 -0.14726211 -0.22665034 -0.36465882 -0.17666844 -0.36392099 -0.01600517 0.36919966 0.15000012 -0.45161461 0.32697005 0.37651586 -0.32509218 0.39219213 0.05086443 -0.38083356 0.24129154 -0.05038089 0.00203199 -0.42874313 0.12536889 -0.3421235 -0.15221594 -0.454692 -0.02656034 -0.02218366 -0.25603051 -0.2249467 -0.07867989 -0.53430609 -0.30063394 0.22210178 0.15167214 -0.11587943 0.14877169 0.025182 -0.11758429 -0.33462337 0.2778078 0.25735238 -0.48218057 -0.25686212 -0.26101231 0.32408123 0.08470069 -0.21274233 -0.21734172 0.00139867 0.15999863 -0.06655995 0.22212123 -0.25234552 0.17242752 0.33072345 -0.48437418 -0.07916299 0.18509961 -0.06689423 0.1527958 -0.16343994 -0.24883281 0.33020765 -0.01695553 -0.64218903 -0.35164878 0.10997594 0.16636878 0.24044841 0.08596227 0.41851186 0.47105082 0.27110334 -0.18742204 -0.34662244 -0.19053041 0.20188182 -0.027384 -0.36669937 -0.2779636 -0.40238442 -0.4008954 -0.31187053 -0.05521224 0.18963317 -0.11824411 -0.37755248 0.19430119 -0.35738383 0.41514174 0.0987715 0.09852836 -0.28925388 -0.04769074 0.26854134 0.11288332 -0.3396557 -0.1980434 -0.15014548 -0.31946746 -0.16966036 0.00285232 0.31716843 0.49169938 -0.16937593 -0.03208759 -0.12566696 0.13767718 0.00668406 0.34061395 -0.30574728 -0.4786323 0.2927631 0.04892921 0.30963658 0.00709893 0.17760978 -0.43836653 0.56808684 0.53985977 0.01376702 0.09184149 0.21105069 0.47836747 -0.65359274 -0.13576892 0.28128686 -0.22935518 0.1405368 -0.09002174 -0.43244135
1,하,하 0.36841672 0.21683283 -0.46029392 0.61336295 -0.01854309 0.27234678 -0.31002707 -0.36359008 -0.37921432 -0.29392107 0.28058396 0.19045135 -0.12515747 -0.47764132 0.20175522 -0.04622723 0.72433827 0.0870152 0.32099495 0.39277455 -0.13883131 0.04784386 0.0090893 0.00469847 0.45565233 -0.15819908 0.23329638 0.29061281 0.52076928 -0.30699805 0.08684634 0.42925093 0.06331168 -0.16192458 -0.21535015 -0.36453321 0.13309419 0.18902064 -0.29920648 -0.31632663 -0.0553658 -0.56913289 -0.17809778 -0.133254 0.2518226 0.27998031 0.20979745 -0.21267827 0.11448084 -0.03507354 0.4688947 0.0764919 -0.08578378 0.50604127 -0.23645624 -0.31395735 0.28556741 -0.38580963 0.0523494 -0.07794431 0.12492694 0.23989632 0.08603445 -0.41798123 0.14097294 0.57576447 -0.38864296 0.36600389 -0.14548736 0.24537953 0.3882426 -0.2656514 -0.41137151 -0.38671694 0.10722915 -0.2155025 0.23774053 -0.08541731 0.14438233 -0.36157824 -0.29063034 0.21515903 -0.56528632 0.4335399 0.33841142 0.2872712 -0.07353999 -0.01709714 0.25221327 -0.39526921 -0.47889734 0.00145772 -0.30913501 -0.16853361 -0.097773 0.12499947 -0.07375591 -0.13768481 0.10251603 -0.55976113 -0.31442303 0.17463187 0.18294199 0.12898277 0.25978554 -0.06468903 0.16384478 -0.26891798 0.17706376 0.29710707 -0.39470815 0.16247905 -0.21252991 0.26779443 0.48325771 -0.02785288 0.21217176 -0.02158545 -0.43185398 0.33519644 0.20739244 -0.3321114 0.17731226 0.30283018 -0.35531105 -0.27099341 -0.0395523 -0.16268168 -0.02659771 0.15538934 -0.34828881 0.21435116 -0.35870617 -0.54089611 -0.39327171 -0.09843882 -0.12970032 0.21336714 0.34533007 -0.00546457 0.44876944 0.42772946 -0.68503217 -0.16522557 0.05959863 0.2565063 0.17919654 -0.21734338 -0.3305915 -0.32107

In [118]:
np.savetxt('../KB_NLP/glove_5290.txt',glove_embedding['vectors'],fmt='%s')